In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Height"
cohort = "GSE101709"

# Input paths
in_trait_dir = "../../input/GEO/Height"
in_cohort_dir = "../../input/GEO/Height/GSE101709"

# Output paths
out_data_file = "../../output/preprocess/Height/GSE101709.csv"
out_gene_data_file = "../../output/preprocess/Height/gene_data/GSE101709.csv"
out_clinical_data_file = "../../output/preprocess/Height/clinical_data/GSE101709.csv"
json_path = "../../output/preprocess/Height/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression analysis of Influenza vaccine response in Young and Old - Year 4"
!Series_summary	"We profiled gene expression from a stratified cohort of subjects to define influenza vaccine response in Young and Old"
!Series_overall_design	"Differential gene expression by human PBMCs from Healthy Adults receiving Influenza Vaccination (Fluvirin, Novartis). Healthy adults (older >65, younger 21-30 years) were recruited at seasonal Influenza Vaccination clinics organized by Yale University Health Services during October to December of 2013 – 2014 seasons. With informed consent, healthy individuals were recruited as per a protocol approved by Human Investigations Committee of the Yale University School of Medicine. Each subject was evaluated by a screening questionnaire determining self-reported demographic information, height, weight, medications and comorbid conditions. Participants with acute illness two weeks prior to vaccination were excluded 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From background information, we see it contains gene expression data from human PBMC samples
# using "Human HT12-V4.0 BeadChip" - this is a gene expression microarray
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For height (our trait), we can see in the background info that height data was collected
# "Each subject was evaluated by a screening questionnaire determining self-reported demographic information, height, weight..."
# However, we don't see height data in the sample characteristics dictionary
trait_row = None  # Height data not available in the sample characteristics

# For age, the sample characteristics dictionary shows age group data at index 1
age_row = 1  # Age group data is available at index 1

# Gender data is not explicitly available in the sample characteristics
gender_row = None  # Gender data not available

# 2.2 Data Type Conversion
# For height (not available, but define a function anyway)
def convert_trait(value):
    if not value or pd.isna(value):
        return None
    
    # Extract the value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Height would typically be a continuous value
        return float(value)
    except (ValueError, TypeError):
        return None

# For age (available as age group)
def convert_age(value):
    if not value or pd.isna(value):
        return None
    
    # Extract the value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert age group to binary (Young=0, Older/Frail=1)
    if 'young' in value.lower():
        return 0
    elif 'older' in value.lower() or 'frail' in value.lower():
        return 1
    else:
        return None

# For gender (not available, but define a function anyway)
def convert_gender(value):
    if not value or pd.isna(value):
        return None
    
    # Extract the value after colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert gender to binary (female=0, male=1)
    if 'female' in value.lower() or 'f' == value.lower():
        return 0
    elif 'male' in value.lower() or 'm' == value.lower():
        return 1
    else:
        return None

# 3. Save Metadata
# The trait data is not available (trait_row is None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction - Skip as trait_row is None


A new JSON file was created at: ../../output/preprocess/Height/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 46892 rows
First 20 gene IDs:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Based on the gene identifiers observed, these are Illumina microarray probe IDs (ILMN_) 
# rather than standard human gene symbols. They need to be mapped to gene symbols for proper analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE101709
Line 6: !Series_title = Gene expression analysis of Influenza vaccine response in Young and Old - Year 4
Line 7: !Series_geo_accession = GSE101709
Line 8: !Series_status = Public on Jan 08 2020
Line 9: !Series_submission_date = Jul 20 2017
Line 10: !Series_last_update_date = Jul 25 2021
Line 11: !Series_pubmed_id = 32060136
Line 12: !Series_summary = We profiled gene expression from a stratified cohort of subjects to define influenza vaccine response in Young and Old
Line 13: !Series_overall_design = Differential gene expression by human PBMCs from Healthy Adults receiving Influenza Vaccination (Fluvirin, Novartis). Healthy adults (older >65, younger 21-30 years) were recruited at seasonal Influen


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain the probe IDs and gene symbols
# From the previous output, we can see:
# - 'ID' column contains the probe IDs (ILMN_*) that match our gene expression data
# - 'Symbol' column contains the gene symbols we need to map to

# 2. Extract the gene mapping dataframe from the gene annotation
probe_id_col = 'ID'
gene_symbol_col = 'Symbol'

# Create the mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"Created gene mapping with {len(gene_mapping)} entries")
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Converted probe-level data to gene-level expression data with {len(gene_data)} genes")
print("First few genes:")
print(gene_data.index[:10])

# Check if we have successfully extracted genes
if gene_data.empty:
    print("Warning: Mapped gene expression data is empty")
    is_gene_available = False
else:
    print(f"Successfully mapped expression data to {len(gene_data.index)} genes")
    is_gene_available = True


Created gene mapping with 44837 entries
Gene mapping preview:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Converted probe-level data to gene-level expression data with 21344 genes
First few genes:
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2',
       'A4GALT', 'A4GNT'],
      dtype='object', name='Gene')
Successfully mapped expression data to 21344 genes


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
import numpy as np
import os

print(f"Gene data before normalization: {gene_data.shape}")

try:
    # Try to normalize gene symbols using the NCBI Gene database
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data after normalization: {normalized_gene_data.shape}")
    
    # If normalization resulted in empty dataframe, use the original gene data
    if normalized_gene_data.empty:
        print("Warning: Normalization resulted in empty gene data. Using original gene data instead.")
        normalized_gene_data = gene_data
        
except Exception as e:
    print(f"Error during gene symbol normalization: {e}")
    print("Using original gene data instead.")
    normalized_gene_data = gene_data

# Save gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Create a minimal clinical dataframe since we don't have trait data
sample_ids = gene_data.columns
minimal_clinical_df = pd.DataFrame(index=sample_ids)
minimal_clinical_df[trait] = np.nan  # Add the trait column with NaN values

# If we have age data from Step 2, add that column
if age_row is not None:
    try:
        minimal_clinical_df['Age'] = get_feature_data(clinical_data, age_row, 'Age', convert_age).iloc[0]
        print("Added Age data to clinical dataframe")
    except Exception as e:
        print(f"Error adding age data: {e}")

# If we have gender data from Step 2, add that column
if gender_row is not None:
    try:
        minimal_clinical_df['Gender'] = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender).iloc[0]
        print("Added Gender data to clinical dataframe")
    except Exception as e:
        print(f"Error adding gender data: {e}")

# Save clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
minimal_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data
try:
    linked_data = pd.concat([minimal_clinical_df, normalized_gene_data.T], axis=1)
    linked_data.index.name = 'Sample'
    print(f"Linked data shape: {linked_data.shape}")
except Exception as e:
    print(f"Error linking clinical and genetic data: {e}")
    # Create a minimal dataframe with just the trait for validation
    linked_data = minimal_clinical_df
    print(f"Using only clinical data with shape: {linked_data.shape}")

# Since trait_row was None in Step 2, we know Height data is not available
is_trait_available = False  

# Add a detailed note about the dataset limitations
note = "Dataset contains gene expression data but no Height measurements. The dataset includes age group information (Young vs Older/Frail) but lacks the specific trait data needed for Height association studies."

# For datasets without trait data, we set is_biased to False
is_biased = False

# Final validation and data quality assessment
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# Only save the linked data if it's usable for our study
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset does not contain Height data and cannot be used for association studies.")

Gene data before normalization: (21344, 98)


Gene data after normalization: (20158, 98)


Gene expression data saved to ../../output/preprocess/Height/gene_data/GSE101709.csv
Added Age data to clinical dataframe
Clinical data saved to ../../output/preprocess/Height/clinical_data/GSE101709.csv
Linked data shape: (98, 20160)
Dataset usability: False
Dataset does not contain Height data and cannot be used for association studies.
